#3 WELCOME TO LANGCHAIN

In [ ]:
#3.0 LLMs and Chat Models
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

# string 기반 질문
chat.predict("How many planets are there?")

In [ ]:
# 3.1 Predict Messages
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

# message 기반 질문
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content="너는 지리학자야"
    ),
    AIMessage(
        content="안녕"
    ),
    HumanMessage(
        content="멕시코와 태국 사이의 거리가 얼마나돼? 그리고 너의 이름은 뭐야?"
    )
]

chat.predict_messages(messages)

In [ ]:
#3.2 prompt template
# 프롬프트 사용

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

# string -> prompt -> LLM
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="Mexico", country_b="Thailand")


chat.predict(prompt)

# message -> prompt -> LLM, 객체가 아닌 string으로 system, ai, human 구분
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)


chat.predict_messages(prompt)


In [ ]:
#3.3 OutputParser and LCEL
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)
from langchain.schema import BaseOutputParser

# BaseOutputParser의 필수 정의 메소드 parse 정의
class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
# 템플릿 정의
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

# 랭체인 표현 언어 | 사용: chain 을 통해, 수행 관계를 보다 간결하게 표현
# CommaOutputParser.parse -> chat.predict_messages -> template(=prompt from messages)
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5, 
    "question": "What are the pokemons?"
})

In [ ]:
#3.4 Chaining Chains ~ #3.5 Recap
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# streaming을 기반으로 답변이 한 글자 단위로 추가되고
# callbacks을 통해 답변의 생성 event를 감지해서 한글자 단위로 print하는 동작을 수행한다.
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 첫번째 chain
chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

# 두번째 chain
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)

veg_chain = veg_chef_prompt | chat

# chain 결합 후 사용
# RunnableMap 개념 사용
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})

#4 MODEL IO

In [ ]:
#4.1 FewShotPromptTemplate
# 예시를 제공함으로써 LLM의 이해를 돕는 방식
# prompt 구성에 예시와 질문이 동일한 계층에서 순서만 다르게 입력된다.

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# key를 미리 정의해 놓은 example
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# exsample이 key로 활용할 template 형식 정의
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

# exsample prompt에 examples의 key들을 차례로 주입하여 FewShot 형성
# suffix을 통해 질문받을 템플릿 구성
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({"country": "Turkey"})


In [ ]:
#4.2 FewShotChatMessagePromptTemplate
# message를 활용하여 system에게 역할 부여 가능
# suffix 를 사용하지 않음(?)

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})

In [ ]:
#4.3 LengthBasedExampleSelector
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# BaseExampleSelector을 사용해서 사용자 정의 selector를 정의
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

# 특정 로직에 따라 examples중에 select하는 과정 수행
example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")



In [ ]:
#4.4 Serialization and Composition
# Serialization이란, 디스크에서 prompt templates를 가져오고 저장하는 방식을 의미 <- 프롬프트 엔지니어와 개발자 사이의 업무 교류를 위해 사용

# 프롬프트 불러오기
# from langchain.prompts import load_prompt
# prompt.format(country="xxx")

# 프롬프트의 memory를 전부 모으는 방법
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# prompt 들 3개 정의
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}
    
    {start}
"""
)

# final와 프롬프트 3개의 관계 설정
prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

# 다중 프롬프트를 final 프롬프트 형식에 맞춰서 종합
full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

# 다중 템플릿의 모든 key를 한번에 정의 가능
chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)


In [ ]:
#4.5 Caching
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# LLM으로 보내는 요청의 횟수를 줄임으로써 절약 수행

# 메모리에 캐싱
# set_llm_cache(InMemoryCache())

# DB에 캐싱
set_llm_cache(SQLiteCache("cache.db"))

# 현재 진행중인 프로세스에 대해 동적으로 출력 수행
# set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

# 프롬프트 -> LLM
chat.predict("How do you make italian pasta")

In [ ]:
# 이전에 받았던 것 반환
chat.predict("How do you make italian pasta")

In [ ]:
#4.6 Serialization
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat = OpenAI(temperature=0.1, max_tokens=450, model="gpt-3.5-turbo-16k")

# LLM 모델 설정 저장
chat.save("model.json")
# LLM 모델 설정 불러오기
loaded_chat = load_llm("model.json")

In [ ]:
# LLM 사용료 미리 특정
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)

#5 MEMORY
- 랭체인은 5가지의 메모리가 존재
- 메모리를 추가하지 않으면 챗봇은 아무것도 기억하지 못함

In [ ]:
#5.0 ConversationBufferMemory
# 대화 내용을 전부 기억하는 메모리. 단, 대화가 길어질수록 메모리 사용 비효율적

from langchain.memory import ConversationBufferMemory

# return_messages=True은 챗모델에서만 설정
# return_messages 설정은 messages 저장 방식을 설정(class or plain text)
memory = ConversationBufferMemory(return_messages=True)

# memory 종류과 관계없이 API 동일
memory.save_context({"input":"Hi!"}, {"output":"How are you"})
memory.load_memory_variables({})

In [ ]:
#5.1 ConversationBufferWindowMemory
# 대화의 특정 부분만을 저장하는 메모리. 최근 대화에만 집중

from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)

memory.save_context({"input":"Hi!"}, {"output":"How are you"})
memory.load_memory_variables({})

In [ ]:
#5.2 ConversationSummaryMemory
# 대화를 요약해서 저장해놓는 메모리. 메모리 관리에 비용이 발생하지만 대화가 길어질 수록 입력 토큰 요구량이 효율적으로 관리됨.

from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# 메모리를 실행하는데에 비용이 든다는 걸 의미
memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("South Kddorea is so pretty", "I wish I could go!!!")
get_history()

In [ ]:
#5.3 ConversationSummaryBufferMemory
# 메모리에 보내온 메시지의 수를 저장. limit에 다다른 순간에 오래된 메시지들을 요약하여 저장
# 버퍼 메모리와 요약 메모리가 결합된 메모리에 해당.

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# max_token_limit은 메시지들이 요약되기 전의 가능한 메시지 토큰 수의 최대값.
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
get_history()

In [ ]:
add_message("South Korea is so pretty", "I wish I could go!!!")
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")
get_history()

In [ ]:
#5.4 ConversationKGMemory
# llm 사용 메모리. 대화 중의 엔티티 knowledge graph 생성. 가장 중요한 것들만 뽑아내는 요약본과 같은 기능 수행

from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
# input을 통해 메모리에서 원하는 정보를 LLM으로 정리해서 추출한다.
memory.load_memory_variables({"input": "who is Nicolas"})


In [ ]:
add_message("Nicolas likes kimchi", "Wow that is so cool!")
memory.load_memory_variables({"inputs": "what does nicolas like"})

In [ ]:
#5.5 Memory on LLMChain
# 메모리를 chain에 꽂는 방법과 두 종류의 chain을 사용해서 꽂는 방법
# LLMchain은 off-the-shelf(일반적인 목적을 가진) chain을 의미한다.

# langchain expression 언어를 활용해서 만든 chain은 프레임워크를 다루느라 머리 싸매거나 off-the-shelf chain을 커스텀하기보다 직접 만들고 싶을 때 사용

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

# 프롬프트 템플릿에 memory를 위한 공간을 만들고
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

# 사용중에 있는 memory class에게 history를 넣을 곳을 알려준다.
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=10,
    memory_key="chat_history",
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is Nico")
chain.predict(question="I live in Seoul")
chain.predict(question="What is my name?")

In [ ]:
#5.6 Chat Based Memory
# memory 클래스를 plain이 아닌 class로 출력할 경우

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

# return_messages=True
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

# MessagesPlaceholder을 사용해서 누가보냈는지 모를 문맥 context를 추가할 자리를 prompt에 만든다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")
chain.predict(question="I live in Seoul")
chain.predict(question="What is my name?")

In [ ]:
#5.7 LCEL Based Memory

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    # memory history 불러오기
    return memory.load_memory_variables({})["history"]

# RunnablePassthrough은 prompt가 format되기 전에 우리가 함수를 실행시키는걸 허락해준다.
# RunnablePassthrough 단계에서 {"question": question} 와 {"history": load_memory의 반환값}을 결합하여 prompt으로 넘겨준다.
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

# invoke 동작에 대한 메모리 저장 수행
def invoke_chain(question):
    result = chain.invoke({"question": question})
    # memory history 저장하기
    memory.save_context(
        # Human
        {"input": question},
        # AI
        {"output": result.content},
    )
    print(result)

# 
invoke_chain("My name is nico")
invoke_chain("What is my name?")

In [ ]:
# 5.8 Recap
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


invoke_chain("My name is nico")
invoke_chain("What is my name?")

"""
강의 내용

- off-the-shelf Chain: 자동으로 LLM으로부터 응답값을 가져오고 memory를 업데이트하는걸 의미

프롬프트에 메모리 추가 방법
1. LLM chain 활용
2. Chat prompt template 활용
3. 수동 메모리 관리 방식
-> 3번 "수동 메모리 관리 방식"을 가장 추천하심
- 메모리를 저장한 후에 메모리로 할 수 있는 것들이 존재함(다른 곳에서 메모리 호출 가능, 데이터베이스를 가지고 있다면 다른 곳에서 데이터베이스 호출 가능)
- LLM이 가져오는 프롬프트에 기록들을 넣는건 "우리의 일"
"""

#6 RAG

In [ ]:
# 6.1 Data Loaders and Splitters
# Data Loader 종류는 매우 많다. 피그마, 페이스북, 슬랙, 파워포인트, ...
# UnstructuredFileLoader을 사용할 경우 다양한 파일을 읽어올 수 있다.
# load_and_split을 사용해서 splitter에 의한 chunk 나누기 수행 가능

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 고려한 점: chunk를 나눌 때, chunk 내의 문맥이 파괴되진 않았는지, chunk size 는 어떤지 , chunk 의 문맥 유지를 위한 overlap 은 얼마나 필요한지, cunk 구분점을 위한 separator "\n" 는 어떻게 설정하는게 좋은지 고려하는 과정이 필요하다
splitter = RecursiveCharacterTextSplitter()

loader = UnstructuredFileLoader("./files/chapter_one.docx")

loader.load_and_split(text_splitter=splitter)

In [ ]:
#6.2 Tiktoken (04:19)
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

# 텍스트 길이를 계산할 때, model과 같은 방법으로 계산하는 게 좋다. 이때, model은 토큰을 기준으로 연산하므로 이를 고려해야 한다.
# from_tiktoken_encoder을 사용하여, model의 토큰 개수 계산을 고려하여 chunk를 나누는 작업을 수행한다.
# tiktoken은 OpenAI에서 만들어진 encoder이다.
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)


loader = UnstructuredFileLoader("./files/chapter_one.docx")

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [4]:
# 입력 내용과 유사한 chunk 들을 vectorDB에서 불러온다
results = vectorstore.similarity_search("where does winston live")

results

[Document(page_content="The Ministry of Love was the really frightening one. There were no windows in it at all. Winston had never been inside the Ministry of Love, nor within half a kilometre of it. It was a place impossible to enter except on official business, and then only by penetrating through a maze of barbed-wire entanglements, steel doors, and hidden machine-gun nests. Even the streets leading up to its outer barriers were roamed by gorilla-faced guards in black uniforms, armed with jointed truncheons.\nWinston turned round abruptly. He had set his features into the expression of quiet optimism which it was advisable to wear when facing the telescreen. He crossed the room into the tiny kitchen. By leaving the Ministry at this time of day he had sacrificed his lunch in the canteen, and he was aware that there was no food in the kitchen except a hunk of dark-coloured bread which had got to be saved for tomorrow's breakfast. He took down from the shelf a bottle of colourless liqu

In [ ]:
#6.5 Langsmith (01:48)

# https://www.langchain.com/langsmith
# .env
# LANGCHAIN_TRACING_V2 = True
# LANGCHAIN_ENDPOINT = "https://api.smith.langchain.com"
# LANGCHAIN_API_KEY = "~Key~"


In [7]:
#6.6 RetrievalQA (18:22)
# LangChain Expression Language(LCEL)
# off-the-shelf chain은 커스텀이 어렵고 업데이트에 따라 소실될 수도 있다
# Stuff(채우기) <- 찾은 문서를 토대로 프롬프트를 채워 사용하는 방안
# Refine(정제, 다듬기) <- model로 질문과 문서를 프롬프트형태로 보낸 후, 받아낸 답변을 다시 문서와 함께 프롬프트형태로 model에 보내 답변을 얻는 과정을 반복하는것
# 즉, Refine의 경우 문서 10개를 참고할 경우 model로 10번 반복해서 답변 생성을 요청하게 된다.
# Map reduce documents chain <- query에 따른 참고 문서들 각각을 model을 사용하여 모두 요약하는 과정 수행. 이후 요약 내용을 종합하여 프롬프트를 구성하여 model에 답변을 요구함. 경우에 따라선, 요약이 아닌 발췌 방식으로 진행되기도 하며, 이러한 발췌는 질문과 관련된 부분을 중심으로 진행됨.
# Map re-rank documents chain <- 각 document들을 사용한 대한 각각의 프롬프트를 구성하여 model로부터 답변을 요구. 받아낸 각 답변에 대해 점수를 매긴 후 가장 높은 점수를 기록한 답변을 최종 반환. 여기서 점수 매기기의 기준과 예시를 프롬프트에 포함시킴으로서 model이 수행하도록 요구함. output parser를 사용하여 최고점수를 받은 것을 반환하도록함

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

# Retrieval란, 구조화되지 않은 쿼리를 해석하여, 문서들을 반환
# VectorStore는 그러한 탐색 과정에서 핵심 역할을 수행할 수 있으나 그게 필요하지 않은 경우 또한 존재하므로 필수는 아니라고 할 수 있다.
# chain_type 의 기본 설정은 stuff
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    # vectorstore에서 as_retriever을 통해 retriever로의 사용이 가능하도록 한다.
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions")


'Victory Mansions is the name of the building where Winston Smith resides. It\'s a run-down apartment complex located in London, the chief city of Airstrip One. The building is described as having a faulty elevator, with the electric current being cut off during daylight hours as part of an economy drive. The hallway of Victory Mansions smells of boiled cabbage and old rag mats. The apartment complex is described as having a gritty, dusty entrance, and Winston\'s flat is located on the seventh floor. At one end of the hallway, there\'s a large colored poster depicting the face of a man with the caption "BIG BROTHER IS WATCHING YOU." The building is grimy and dilapidated, with a meagre and worn-down appearance.'

In [ ]:
# 이유는 불분명하지만, FAISS와 Chroma VectorStore 중 FAISS의 성능이 더 우수하다고 함.

In [9]:
#6.7~6.8 Stuff LCEL Chain (10:13)
# LCEL 구조 docs: https://python.langchain.com/v0.1/docs/expression_language/interface/
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

# retriver: query string -> list of chunk. 이건 context 와 같은 속성에 넣어 prompt에서 사용
# prompt: template에 정의된 속성들 입력 -> 프롬프트
# llm: 프롬프트 -> 답변
chain = (
    # RunnableParallel에 의해 해당 중괄호 내의 작업은 병렬로 처리된다. 즉, 많은 속성을 동시에 사용하더라도 해당 과정의 작업 속도가 크게 느려지진 않는다.
    {
        # 여기서 retriver은 query에 의한 chunk 탐색과정을 수행한 후 list를 반환함. retriver(query)가 생략된 것과 동일
        "context": retriver,
        # 질문을 전달받음. RunnablePassthrough은 chain의 메소드에 의해 전달된 인자를 반환해주는 동작을 하는 것으로 추측됨.
        "question": RunnablePassthrough(),
    }
    # prompt.format_messages(context=retriver(query),question=RunnablePassthrough())가 생략된 것과 동일.
    | prompt
    | llm
)

chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a building where Winston Smith lives. It is described as having glass doors that let in gritty dust, a hallway that smells of boiled cabbage and old rag mats, and a poster of a large face with the caption "BIG BROTHER IS WATCHING YOU." The building has a faulty lift, and Winston, who is thirty-nine with a varicose ulcer, lives on the seventh floor. The building is part of the city of London in Airstrip One, a province of Oceania.')

#7 DocumentGPT

In [ ]:
#7.0 Introduction (05:01)
# Home.py 파일 <- streamlit run Home.py를 수행함으로써 UI 활성화
import streamlit as st

st.title("Hello world!")

st.subheader("Welcome to Streamlit!")

st.markdown(
    """
    #### I love it!
"""
)

In [ ]:
#7.1 Magic (06:31)
import streamlit as st


st.selectbox(
    "Choose your model",
    (
        "GPT-3",
        "GPT-4",
    ),
)


In [ ]:
#7.2 Data Flow (06:18)
# 수정사항 발생할 때마다 모든 코드가 다시 실행되는 data flow를 지니고 있음
# 단순히 selectbox를 변경할 때에도 수정사항으로 취급하여 전체 코드가 재실행됨
# 물론, Streamlit 내부에서 기본적으로 재실행되지 않는 부분 또한 존재
import streamlit as st
from datetime import datetime

# 이 코드는, 해당 코드를 실행시킨 순간의 시간을 변수에 string으로 담는 것이기에, 시간이 지남에 따른 자동 업데이트 기능은 없다.
# 단순히 코드 재실행 여부를 알려주기 위해 존재하는 코드이다.
today = datetime.today().strftime("%H:%M:%S")

st.title(today)


model = st.selectbox(
    "Choose your model",
    (
        "GPT-3",
        "GPT-4",
    ),
)

if model == "GPT-3":
    st.write("cheap")
else:
    st.write("not cheap")
    name = st.text_input("What is your name?")
    st.write(name)

    value = st.slider(
        "temperature",
        min_value=0.1,
        max_value=1.0,
    )

    st.write(value)


In [ ]:
#7.3 Multi Page

# Home.py
import streamlit as st

st.set_page_config(
    page_title="FullstackGPT Home",
    page_icon="🤖",
)

# 여기서 /DocumentGPT 은 pages/01_DocumentGPT.py 로 이동시킨다.
st.markdown(
    """
# Hello!
            
Welcome to my FullstackGPT Portfolio!
            
Here are the apps I made:
            
- [ ] [DocumentGPT](/DocumentGPT)
- [ ] [PrivateGPT](/PrivateGPT)
- [ ] [QuizGPT](/QuizGPT)
- [ ] [SiteGPT](/SiteGPT)
- [ ] [MeetingGPT](/MeetingGPT)
- [ ] [InvestorGPT](/InvestorGPT)
"""
)

In [ ]:
# pages/01_DocumentGPT.py

# 여기서 01_은 sidebar에서의 순서를 위해 지정한 것. sidebar에서 해당 숫자가 보이진 않음
# 페이지 네비게이션를 위해선 pages 폴더가 필수 
import streamlit as st

st.title("DocumentGPT")


In [ ]:
#7.4 Chat Messages

# pages/01_DocumentGPT.py
import time
import streamlit as st

st.set_page_config(
    page_title="DocumentGPT",
    page_icon="📃",
)

st.title("DocumentGPT")

# 대화 내용을 저장할 공간 마련
if "messages" not in st.session_state:
    st.session_state["messages"] = []


def send_message(message, role, save=True):
    with st.chat_message(role):
        st.write(message)
    if save:
        # 대화 내용 저장
        st.session_state["messages"].append({"message": message, "role": role})


# 대화 내용 전부 불러오기
for message in st.session_state["messages"]:
    send_message(
        message["message"],
        message["role"],
        save=False,
    )


message = st.chat_input("Send a message to the ai ")

if message:
    send_message(message, "human")
    time.sleep(2)
    send_message(f"You said: {message}", "ai")

    with st.sidebar:
        st.write(st.session_state)


In [ ]:
#7.6 Uploading Documents 
# cache를 사용해서 업로드한 documents에 대한 기록과 임베딩 정보를 기록
# .cache 폴더 내에 files과 embeddings 폴더를 생성해둘 것

# pages/01_DocumentGPT.py
import time
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import CacheBackedEmbeddings, OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
import streamlit as st

st.set_page_config(
    page_title="DocumentGPT",
    page_icon="📃",
)


def embed_file(file):
    file_content = file.read()
    file_path = f"./.cache/files/{file.name}"
    with open(file_path, "wb") as f:
        f.write(file_content)
    cache_dir = LocalFileStore(f"./.cache/embeddings/{file.name}")
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=600,
        chunk_overlap=100, 
    )
    loader = UnstructuredFileLoader(file_path)
    docs = loader.load_and_split(text_splitter=splitter)
    embeddings = OpenAIEmbeddings()
    cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
    vectorstore = FAISS.from_documents(docs, cached_embeddings)
    retriever = vectorstore.as_retriever()
    return retriever


st.title("DocumentGPT")

st.markdown(
    """
Welcome!
            
Use this chatbot to ask questions to an AI about your files!
"""
)

file = st.file_uploader(
    "Upload a .txt .pdf or .docx file",
    type=["pdf", "txt", "docx"],
)

if file:
    retriever = embed_file(file)
    s = retriever.invoke("winston")
    s


In [ ]:
# streamlit에서 환경변수 설정하기
# .streamlit/secrets.toml

# OPENAI_API_KEY="~key~"

# LANGCHAIN_TRACING_V2 = true
# LANGCHAIN_ENDPOINT = "https://api.smith.langchain.com"
# LANGCHAIN_API_KEY = "~key~"

In [ ]:
#7.7 Chat History


# pages/01_DocumentGPT.py
import time
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import CacheBackedEmbeddings, OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
import streamlit as st

st.set_page_config(
    page_title="DocumentGPT",
    page_icon="📃",
)

# @st.cache_data(show_spinner="Embedding file...")을 달 경우,
# 함수에 넘긴 input이 이전에 넘겼던 input과 동일할 경우, 완전히 동일한 함수를 재실행하는 것으로 판단하여
# Streamlit은 해당 함수가 그 input을 를 받았을 때 반환했던 값을 cach에서 가져와서 그대로 반환한 후 종료한다.
# 여기서 input 동일성 여부를 판단함에 있어서, hash를 사용한다.
@st.cache_data(show_spinner="Embedding file...")
def embed_file(file):
    file_content = file.read()
    file_path = f"./.cache/files/{file.name}"
    with open(file_path, "wb") as f:
        f.write(file_content)
    cache_dir = LocalFileStore(f"./.cache/embeddings/{file.name}")
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=600,
        chunk_overlap=100,
    )
    loader = UnstructuredFileLoader(file_path)
    docs = loader.load_and_split(text_splitter=splitter)
    embeddings = OpenAIEmbeddings()
    cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
    vectorstore = FAISS.from_documents(docs, cached_embeddings)
    retriever = vectorstore.as_retriever()
    return retriever


def send_message(message, role, save=True):
    with st.chat_message(role):
        st.markdown(message)
    if save:
        st.session_state["messages"].append({"message": message, "role": role})


def paint_history():
    for message in st.session_state["messages"]:
        send_message(
            message["message"],
            message["role"],
            save=False,
        )


st.title("DocumentGPT")

st.markdown(
    """
Welcome!
            
Use this chatbot to ask questions to an AI about your files!

Upload your files on the sidebar.
"""
)

with st.sidebar:
    file = st.file_uploader(
        "Upload a .txt .pdf or .docx file",
        type=["pdf", "txt", "docx"],
    )

if file:
    retriever = embed_file(file)
    send_message("I'm ready! Ask away!", "ai", save=False)
    paint_history()
    message = st.chat_input("Ask anything about your file...")
    if message:
        send_message(message, "human")
        send_message(f"your messages: {message}", "ai")
else:
    st.session_state["messages"] = []


In [ ]:
#7.8 Chain


# pages/01_DocumentGPT.py
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import CacheBackedEmbeddings, OpenAIEmbeddings
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models import ChatOpenAI
import streamlit as st

st.set_page_config(
    page_title="DocumentGPT",
    page_icon="📃",
)

llm = ChatOpenAI(
    temperature=0.1,
)


@st.cache_data(show_spinner="Embedding file...")
def embed_file(file):
    file_content = file.read()
    file_path = f"./.cache/files/{file.name}"
    with open(file_path, "wb") as f:
        f.write(file_content)
    cache_dir = LocalFileStore(f"./.cache/embeddings/{file.name}")
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=600,
        chunk_overlap=100,
    )
    loader = UnstructuredFileLoader(file_path)
    docs = loader.load_and_split(text_splitter=splitter)
    embeddings = OpenAIEmbeddings()
    cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
    vectorstore = FAISS.from_documents(docs, cached_embeddings)
    retriever = vectorstore.as_retriever()
    return retriever


def send_message(message, role, save=True):
    with st.chat_message(role):
        st.markdown(message)
    if save:
        st.session_state["messages"].append({"message": message, "role": role})


def paint_history():
    for message in st.session_state["messages"]:
        send_message(
            message["message"],
            message["role"],
            save=False,
        )


def format_docs(docs):
    return "\n\n".join(document.page_content for document in docs)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Answer the question using ONLY the following context. If you don't know the answer just say you don't know. DON'T make anything up.
            
            Context: {context}
            """,
        ),
        ("human", "{question}"),
    ]
)


st.title("DocumentGPT")

st.markdown(
    """
Welcome!
            
Use this chatbot to ask questions to an AI about your files!

Upload your files on the sidebar.
"""
)

with st.sidebar:
    file = st.file_uploader(
        "Upload a .txt .pdf or .docx file",
        type=["pdf", "txt", "docx"],
    )

# retriever | RunnableLambda(format_docs)은 format_docs(retriever.invoke(query))와 의미가 같다.
if file:
    retriever = embed_file(file)
    send_message("I'm ready! Ask away!", "ai", save=False)
    paint_history()
    message = st.chat_input("Ask anything about your file...")
    if message:
        send_message(message, "human")
        chain = (
            {
                "context": retriever | RunnableLambda(format_docs),
                "question": RunnablePassthrough(),
            }
            | prompt
            | llm
        )
        response = chain.invoke(message)
        send_message(response.content, "ai")

else:
    st.session_state["messages"] = []


In [ ]:
#7.9 Streaming

# ChatCallbackHandler 사용
# pages/01_DocumentGPT.py
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import CacheBackedEmbeddings, OpenAIEmbeddings
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.base import BaseCallbackHandler
import streamlit as st

st.set_page_config(
    page_title="DocumentGPT",
    page_icon="📃",
)

# 오래된 llm 모델에서는 간혹 지원하지 않을 수 있다.
# 또한, Callback Handler는 event들을 listen하는 여러 function들을 가진다.
class ChatCallbackHandler(BaseCallbackHandler):
    message = ""

    # args로 on_llm_start(1,2,3)등의 입력을 받을 수 있으며,
    # kwargs로 on_llm_start(a=1, a=4, b=1)등의 입력을 받을 수 있다.
    # llm을 초기화한다. message_box을 통해 스트리밍을 진행하고자한다.
    def on_llm_start(self, *args, **kwargs):
        # 생성 token에 대해 stream 형식으로 화면에 출력하기 위해 사용
        self.message_box = st.empty()

    # llm 답변을 저장한다.
    def on_llm_end(self, *args, **kwargs):
        save_message(self.message, "ai")

    # 답변이 생성되는 과정(토큰 단위)에 따라 스트리밍된다.
    def on_llm_new_token(self, token, *args, **kwargs):
        self.message += token
        self.message_box.markdown(self.message)


llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        ChatCallbackHandler(),
    ],
)


@st.cache_data(show_spinner="Embedding file...")
def embed_file(file):
    file_content = file.read()
    file_path = f"./.cache/files/{file.name}"
    with open(file_path, "wb") as f:
        f.write(file_content)
    cache_dir = LocalFileStore(f"./.cache/embeddings/{file.name}")
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=600,
        chunk_overlap=100,
    )
    loader = UnstructuredFileLoader(file_path)
    docs = loader.load_and_split(text_splitter=splitter)
    embeddings = OpenAIEmbeddings()
    cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
    vectorstore = FAISS.from_documents(docs, cached_embeddings)
    retriever = vectorstore.as_retriever()
    return retriever


def save_message(message, role):
    st.session_state["messages"].append({"message": message, "role": role})


def send_message(message, role, save=True):
    with st.chat_message(role):
        st.markdown(message)
    if save:
        save_message(message, role)


def paint_history():
    for message in st.session_state["messages"]:
        send_message(
            message["message"],
            message["role"],
            save=False,
        )


def format_docs(docs):
    return "\n\n".join(document.page_content for document in docs)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Answer the question using ONLY the following context. If you don't know the answer just say you don't know. DON'T make anything up.
            
            Context: {context}
            """,
        ),
        ("human", "{question}"),
    ]
)


st.title("DocumentGPT")

st.markdown(
    """
Welcome!
            
Use this chatbot to ask questions to an AI about your files!

Upload your files on the sidebar.
"""
)

with st.sidebar:
    file = st.file_uploader(
        "Upload a .txt .pdf or .docx file",
        type=["pdf", "txt", "docx"],
    )

if file:
    retriever = embed_file(file)
    send_message("I'm ready! Ask away!", "ai", save=False)
    paint_history()
    message = st.chat_input("Ask anything about your file...")
    if message:
        send_message(message, "human")
        chain = (
            {
                "context": retriever | RunnableLambda(format_docs),
                "question": RunnablePassthrough(),
            }
            | prompt
            | llm
        )
        # ai가 답변을 작성하고 있는 걸 표현하기 위해 사용
        with st.chat_message("ai"):
            response = chain.invoke(message)


else:
    st.session_state["messages"] = []



In [ ]:
#7.10 Recap

# pages/01_DocumentGPT.py
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import CacheBackedEmbeddings, OpenAIEmbeddings
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.base import BaseCallbackHandler
import streamlit as st

st.set_page_config(
    page_title="DocumentGPT",
    page_icon="📃",
)


class ChatCallbackHandler(BaseCallbackHandler):
    message = ""

    def on_llm_start(self, *args, **kwargs):
        self.message_box = st.empty()

    def on_llm_end(self, *args, **kwargs):
        save_message(self.message, "ai")

    def on_llm_new_token(self, token, *args, **kwargs):
        self.message += token
        self.message_box.markdown(self.message)


llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        ChatCallbackHandler(),
    ],
)


@st.cache_data(show_spinner="Embedding file...")
def embed_file(file):
    file_content = file.read()
    file_path = f"./.cache/files/{file.name}"
    with open(file_path, "wb") as f:
        f.write(file_content)
    cache_dir = LocalFileStore(f"./.cache/embeddings/{file.name}")
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=600,
        chunk_overlap=100,
    )
    loader = UnstructuredFileLoader(file_path)
    docs = loader.load_and_split(text_splitter=splitter)
    embeddings = OpenAIEmbeddings()
    cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
    vectorstore = FAISS.from_documents(docs, cached_embeddings)
    retriever = vectorstore.as_retriever()
    return retriever


def save_message(message, role):
    st.session_state["messages"].append({"message": message, "role": role})


def send_message(message, role, save=True):
    with st.chat_message(role):
        st.markdown(message)
    if save:
        save_message(message, role)


def paint_history():
    for message in st.session_state["messages"]:
        send_message(
            message["message"],
            message["role"],
            save=False,
        )


def format_docs(docs):
    return "\n\n".join(document.page_content for document in docs)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Answer the question using ONLY the following context. If you don't know the answer just say you don't know. DON'T make anything up.
            
            Context: {context}
            """,
        ),
        ("human", "{question}"),
    ]
)


st.title("DocumentGPT")

st.markdown(
    """
Welcome!
            
Use this chatbot to ask questions to an AI about your files!

Upload your files on the sidebar.
"""
)

with st.sidebar:
    file = st.file_uploader(
        "Upload a .txt .pdf or .docx file",
        type=["pdf", "txt", "docx"],
    )

if file:
    retriever = embed_file(file)
    send_message("I'm ready! Ask away!", "ai", save=False)
    paint_history()
    message = st.chat_input("Ask anything about your file...")
    if message:
        send_message(message, "human")
        chain = (
            {
                "context": retriever | RunnableLambda(format_docs),
                "question": RunnablePassthrough(),
            }
            | prompt
            | llm
        )
        with st.chat_message("ai"):
            chain.invoke(message)


else:
    st.session_state["messages"] = []



In [ ]:
# 위의 코드에서 session_state가 아닌 memory으로 전환 가능

#8 PrivateGPT

In [ ]:
#8.0 Introduction (04:14)

# pages/02_PrivateGPT.py
# .cache 폴더 내의 files과 embeddings 폴더가 겹칠 수 있으므로,
# 서로 다른 embedding 모델을 사용했다는 걸 고려하여 구분 가능하도록 폴더를 추가한다.
# ex: .cache/private_files, .cache/private_embeddings
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import CacheBackedEmbeddings, OpenAIEmbeddings
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.base import BaseCallbackHandler
import streamlit as st

st.set_page_config(
    page_title="PrivateGPT",
    page_icon="📃",
)


class ChatCallbackHandler(BaseCallbackHandler):
    message = ""

    def on_llm_start(self, *args, **kwargs):
        self.message_box = st.empty()

    def on_llm_end(self, *args, **kwargs):
        save_message(self.message, "ai")

    def on_llm_new_token(self, token, *args, **kwargs):
        self.message += token
        self.message_box.markdown(self.message)


llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        ChatCallbackHandler(),
    ],
)


@st.cache_data(show_spinner="Embedding file...")
def embed_file(file):
    file_content = file.read()
    file_path = f"./.cache/private_files/{file.name}"
    with open(file_path, "wb") as f:
        f.write(file_content)
    cache_dir = LocalFileStore(f"./.cache/private_embeddings/{file.name}")
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=600,
        chunk_overlap=100,
    )
    loader = UnstructuredFileLoader(file_path)
    docs = loader.load_and_split(text_splitter=splitter)
    embeddings = OpenAIEmbeddings()
    cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
    vectorstore = FAISS.from_documents(docs, cached_embeddings)
    retriever = vectorstore.as_retriever()
    return retriever


def save_message(message, role):
    st.session_state["messages"].append({"message": message, "role": role})


def send_message(message, role, save=True):
    with st.chat_message(role):
        st.markdown(message)
    if save:
        save_message(message, role)


def paint_history():
    for message in st.session_state["messages"]:
        send_message(
            message["message"],
            message["role"],
            save=False,
        )


def format_docs(docs):
    return "\n\n".join(document.page_content for document in docs)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Answer the question using ONLY the following context. If you don't know the answer just say you don't know. DON'T make anything up.
            
            Context: {context}
            """,
        ),
        ("human", "{question}"),
    ]
)


st.title("PrivateGPT")

st.markdown(
    """
Welcome!
            
Use this chatbot to ask questions to an AI about your files!

Upload your files on the sidebar.
"""
)

with st.sidebar:
    file = st.file_uploader(
        "Upload a .txt .pdf or .docx file",
        type=["pdf", "txt", "docx"],
    )

if file:
    retriever = embed_file(file)
    send_message("I'm ready! Ask away!", "ai", save=False)
    paint_history()
    message = st.chat_input("Ask anything about your file...")
    if message:
        send_message(message, "human")
        chain = (
            {
                "context": retriever | RunnableLambda(format_docs),
                "question": RunnablePassthrough(),
            }
            | prompt
            | llm
        )
        with st.chat_message("ai"):
            chain.invoke(message)


else:
    st.session_state["messages"] = []


In [ ]:
#8.1 HuggingFaceHub

# model inference api 사용하기

# Instruct model의 경우 template에 대한 구체적인 가이드라인이 정해져 있다. 이를 고려하여 API 요청을 보내는 것이 중요하다.
# 이는 해당 모델을 직접 다운로드 받아 사용할 때에도 동일한 이슈일 것으로 판단된다.
# 구체적인 관련 사항은 해당 모델을 업로드한 기업 홈페이지의 model documentation을 확인하는게 중요하다.
# 또한, 허깅페이스에서 max new token 설정 범위 또한 알아야 한다.
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("[INST]What is the meaning of {word}[/INST]")

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={
        "max_new_tokens": 250,
    },
)

chain = prompt | llm

chain.invoke({"word": "potato"})


In [ ]:
#8.2 HuggingFacePipeline

# model 다운 후 로컬에서 돌리기
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("A {word} is a")


llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    # task는 text-generation, text2text-generation, summarization 등 존재. 허깅페이스 repo에 특화 task가 서술되어 있음
    task="text-generation",
    # device=0로 설정 시에 GPU로 동작. device=-1로 설정 시에는 CPU로 동작.
    pipeline_kwargs={"max_new_tokens": 150},
)

chain = prompt | llm

chain.invoke({"word": "tomato"})

In [ ]:
#8.3 GPT4All

# 로컬에서 model 실행하는 또 다른 방법
# 실제 로컬 모델과 함께 사용할 수 있는 UI 애플리케이션을 갖고 있음
# fine tuning을 할 수 있는 많은 모델들을 유지하고 있음
# GPT4All 홈페이지에서 모델을 다운받아와야 사용 가능
from langchain.llms.gpt4all import GPT4All
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "You are a helpful assistant that defines words. Define this word: {word}."
)

llm = GPT4All(
    model="./falcon.bin",
)

chain = prompt | llm

chain.invoke({"word": "tomato"})


In [ ]:
#8.4 Ollama

# llm을 로컬적으로 사용하는 방식 
# pages/02_PrivateGPT.py
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import CacheBackedEmbeddings, OllamaEmbeddings
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models import ChatOllama
from langchain.callbacks.base import BaseCallbackHandler
import streamlit as st

st.set_page_config(
    page_title="PrivateGPT",
    page_icon="📃",
)


class ChatCallbackHandler(BaseCallbackHandler):
    message = ""

    def on_llm_start(self, *args, **kwargs):
        self.message_box = st.empty()

    def on_llm_end(self, *args, **kwargs):
        save_message(self.message, "ai")

    def on_llm_new_token(self, token, *args, **kwargs):
        self.message += token
        self.message_box.markdown(self.message)


llm = ChatOllama(
    model="mistral:latest",
    temperature=0.1,
    streaming=True,
    callbacks=[
        ChatCallbackHandler(),
    ],
)


@st.cache_data(show_spinner="Embedding file...")
def embed_file(file):
    file_content = file.read()
    file_path = f"./.cache/private_files/{file.name}"
    with open(file_path, "wb") as f:
        f.write(file_content)
    cache_dir = LocalFileStore(f"./.cache/private_embeddings/{file.name}")
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=600,
        chunk_overlap=100,
    )
    loader = UnstructuredFileLoader(file_path)
    docs = loader.load_and_split(text_splitter=splitter)
    embeddings = OllamaEmbeddings(model="mistral:latest")
    cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
    vectorstore = FAISS.from_documents(docs, cached_embeddings)
    retriever = vectorstore.as_retriever()
    return retriever


def save_message(message, role):
    st.session_state["messages"].append({"message": message, "role": role})


def send_message(message, role, save=True):
    with st.chat_message(role):
        st.markdown(message)
    if save:
        save_message(message, role)


def paint_history():
    for message in st.session_state["messages"]:
        send_message(
            message["message"],
            message["role"],
            save=False,
        )


def format_docs(docs):
    return "\n\n".join(document.page_content for document in docs)


prompt = ChatPromptTemplate.from_template(
    """Answer the question using ONLY the following context and not your training data. If you don't know the answer just say you don't know. DON'T make anything up.
    
    Context: {context}
    Question:{question}
    """
)


st.title("PrivateGPT")

st.markdown(
    """
Welcome!
            
Use this chatbot to ask questions to an AI about your files!

Upload your files on the sidebar.
"""
)

with st.sidebar:
    file = st.file_uploader(
        "Upload a .txt .pdf or .docx file",
        type=["pdf", "txt", "docx"],
    )

if file:
    retriever = embed_file(file)
    send_message("I'm ready! Ask away!", "ai", save=False)
    paint_history()
    message = st.chat_input("Ask anything about your file...")
    if message:
        send_message(message, "human")
        chain = (
            {
                "context": retriever | RunnableLambda(format_docs),
                "question": RunnablePassthrough(),
            }
            | prompt
            | llm
        )
        with st.chat_message("ai"):
            chain.invoke(message)


else:
    st.session_state["messages"] = []


integration